In [1]:
# import package
from datetime import datetime as dtime
from datetime import date,timedelta
from urllib.request import urlopen
import requests
from dateutil import rrule
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import json
import time
import os
from io import StringIO
from Tools import *

In [2]:
#讀取INI檔案
InitFile = IniTool()

Load init file: .\Initialization.ini


In [3]:
#轉換日期民國到西元
def date_convert(strDate):
    arydate = strDate.split("/")
    return date(int(arydate[0]) + 1911, int(arydate[1]), int(arydate[2])).strftime('%Y/%m/%d')

In [4]:
# 爬取每月股價的目標網站並包裝成函式
def craw_one_month(stock_number,date):
    url = (
        "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+
        date.strftime('%Y%m%d')+
        "&stockNo="+
        str(stock_number)
    )
    data = json.loads(urlopen(url).read())
    return pd.DataFrame(data['data'],columns=data['fields'])

In [5]:
# 根據使用者輸入的日期，以月為單位，重複呼叫爬取月股價的函式
def craw_stock(stock_number, start_month):
    b_month = date(*[int(x) for x in start_month.split('-')])
    now = dtime.now().strftime("%Y-%m-%d")  # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    
    result = pd.DataFrame()
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        print(dt)
        result = pd.concat([result, craw_one_month(stock_number, dt)], ignore_index=True)
        time.sleep(10000.0 / 1000.0);

    return result

In [6]:
def ConvertType_NewData(df):
    #轉換日期民國到西元.
    df['日期'] = [date_convert(x) for x in df['日期']]
    df['日期'] = pd.to_datetime(df['日期'].astype(str), format='%Y/%m/%d')
    #轉換型態.
    if df['成交筆數'].dtype != 'float64':
        df['成交筆數'] = df['成交筆數'].str.replace(',','').astype(float)
    if df['成交股數'].dtype != 'float64':
        df['成交股數'] = df['成交股數'].str.replace(',','').astype(float)
    if df['成交金額'].dtype != 'float64':
        df['成交金額'] = df['成交金額'].str.replace(',','').astype(float)

In [7]:
# 建立資料夾
def MKDir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)

In [8]:
NoHistory = False

In [9]:
DataPath = InitFile.Read('Infomation', 'DataPath')
Code = InitFile.Read('Infomation', 'StockCode')
Date = InitFile.Read('Infomation', 'StartDate')
print(Code)
print(Date)
print(DataPath)
fullpath = DataPath + Code + '.csv'
print(fullpath) 

2303
2020-01-01
./Data/
./Data/2303.csv


In [10]:
#確認是否有舊資料
if os.path.isfile(fullpath):
    #檔案存在的處理
    print("檔案存在。")
else:
    #檔案不存在的處理
    print("檔案不存在。")
    NoHistory = True

檔案存在。


In [11]:
df_History = pd.DataFrame()

In [12]:
if NoHistory is True:
    df_History = craw_stock(Code,Date)

In [41]:
#測試用變數
df_His = df_History.copy()

In [42]:
if NoHistory is True:
    #轉換日期民國到西元.
    df_His['日期'] = [date_convert(x) for x in df_His['日期']]
    df_His['日期'] = pd.to_datetime(df_His['日期'].astype(str), format='%Y/%m/%d')
    #轉換型態.
    if df_His['成交筆數'].dtype != 'float64':
        df_His['成交筆數'] = df_His['成交筆數'].str.replace(',','').astype(float)
    if df_His['成交股數'].dtype != 'float64':
        df_His['成交股數'] = df_His['成交股數'].str.replace(',','').astype(float)
    if df_His['成交金額'].dtype != 'float64':
        df_His['成交金額'] = df_His['成交金額'].str.replace(',','').astype(float)

In [43]:
df_His.to_csv(fullpath, encoding='utf_8_sig')

In [44]:
df_His.dtypes

Series([], dtype: object)

In [17]:
#清空
df_His = pd.DataFrame()

In [18]:
df_His = pd.read_csv(fullpath, index_col = 0)
df_His

""


In [19]:
df_His.dtypes

Series([], dtype: object)

In [40]:
#轉換日期格式.
df_His['日期'] = pd.to_datetime(df_His['日期'].astype(str), format='%Y/%m/%d')

KeyError: '日期'

In [21]:
df_His.dtypes

Series([], dtype: object)

In [22]:
#取得今天日期
today = date.today()

In [45]:
#取的今天的表格
df_Today = craw_one_month(Code,today)
# type(Code)
# str(Code)

In [46]:
df_Today.dtypes

日期      object
成交股數    object
成交金額    object
開盤價     object
最高價     object
最低價     object
收盤價     object
漲跌價差    object
成交筆數    object
dtype: object

In [25]:
ConvertType_NewData(df_Today)

In [47]:
#找出歷史的最後一天和目前的最後一天進行比較
lastday_his = df_His.loc[:,'日期'].max()
lastday_his

KeyError: '日期'

In [27]:
lastday_today = df_Today.loc[:,'日期'].max()
lastday_today

Timestamp('2021-07-09 00:00:00')

In [ ]:
between.days

In [28]:
between = lastday_today - lastday_his

NameError: name 'lastday_his' is not defined

In [29]:
lastday_his2 = lastday_his - timedelta(days=lastday_his.day-1)
lastday_his2

NameError: name 'lastday_his' is not defined

In [30]:
df_Today.loc[df_Today['日期'] > lastday_his]

NameError: name 'lastday_his' is not defined

In [31]:
df_His

""


In [32]:
#合併資料
result = pd.DataFrame()
result = pd.concat([result, df_His], ignore_index=True)
result = pd.concat([result, df_Today.loc[df_Today['日期'] > lastday_his]], ignore_index=True)
result

NameError: name 'lastday_his' is not defined

In [33]:
df

NameError: name 'df' is not defined

In [34]:
#複製測試用
df2 = df.copy()
df2

NameError: name 'df' is not defined

In [35]:
df2['日期'] = pd.to_datetime(df2['日期'].astype(str), format='%Y/%m/%d')

NameError: name 'df2' is not defined

In [36]:
#轉換型態.
if df2['成交筆數'].dtype != 'float64':
    df2['成交筆數'] = df2['成交筆數'].str.replace(',','').astype(float)
if df2['成交股數'].dtype != 'float64':
    df2['成交股數'] = df2['成交股數'].str.replace(',','').astype(float)
if df2['成交金額'].dtype != 'float64':
    df2['成交金額'] = df2['成交金額'].str.replace(',','').astype(float)

NameError: name 'df2' is not defined

In [37]:
df2
df2.dtypes

NameError: name 'df2' is not defined

In [39]:
df2.loc[:,'日期'].max()

NameError: name 'df2' is not defined

In [ ]:
df2.loc[df2["日期"]==df2.loc[:,"日期"].max(),"日期"]

In [ ]:
#讀取原有的資料.
df_History = pd.read_csv(fullpath)

In [ ]:
df_History.dtypes
df_History

In [ ]:
#轉換型態.
df_History['日期'] = pd.to_datetime(df_History['日期'].astype(str), format='%Y/%m/%d')
if df_History['成交筆數'].dtype != 'float64':
    df_History['成交筆數'] = df_History['成交筆數'].str.replace(',','').astype(float)
if df_History['成交股數'].dtype != 'float64':
    df_History['成交股數'] = df_History['成交股數'].str.replace(',','').astype(float)
if df_History['成交金額'].dtype != 'float64':
    df_History['成交金額'] = df_History['成交金額'].str.replace(',','').astype(float)